## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a coversation and remember previous interactions.

Note that this chatbot that we build will only use language model to have a conversation. There are several other related concepts that you may be looking for:
- Conversational RAG: Enable a chatbot experience over an external source of data.
- Agents: Build a chatbot that can take actions

This will cover the basics which will be helpful for those two more advanced topics.

In [33]:
import os
from dotenv import load_dotenv
load_dotenv() ## adding all the environment variable

groq_api_key = os.getenv('GROQ_API_KEY')


In [34]:
from langchain_groq import ChatGroq
model = ChatGroq(model ='Gemma2-9b-It',groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E596B952E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E596BA5A90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [35]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My Name is Nageshwar and I am a software engineer.")])

AIMessage(content="Hi Nageshwar, it's nice to meet you! 👋\n\nBeing a software engineer is awesome. What kind of software do you enjoy working on?  Do you have any current projects you're excited about? 😄  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 24, 'total_tokens': 75, 'completion_time': 0.092727273, 'prompt_time': 0.00133434, 'queue_time': 0.25557902, 'total_time': 0.094061613}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--81986994-7ec3-407e-bf5b-41ba95fe2590-0', usage_metadata={'input_tokens': 24, 'output_tokens': 51, 'total_tokens': 75})

In [36]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My Name is Nageshwar and I am a software engineer."),
    AIMessage(content="Hello Nageshwar, it's nice to meet you!\n\nIt's great to know you're a software engineer. What kind of software development are you involved in? Do you have a particular area of expertise or a project you're working on that you'd like to talk about? I'm always interested in learning more about what people are doing in the world of software.\n"),
    HumanMessage(content="Hey what's my name and what do I do?")
])

AIMessage(content="You told me your name is Nageshwar and that you are a software engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 128, 'total_tokens': 169, 'completion_time': 0.074545455, 'prompt_time': 0.00338356, 'queue_time': 0.255496929, 'total_time': 0.077929015}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ac9ce3dd-9939-4134-b1ce-e8ebdc32ba0e-0', usage_metadata={'input_tokens': 128, 'output_tokens': 41, 'total_tokens': 169})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the models, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of input

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history= get_session_history)

In [38]:
config = {"configurable":{"session_id":"chat1"}}


In [39]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My Name is Nageshwar and I am a software engineer.")],
    config= config
)

In [40]:
response.content

"Hi Nageshwar, it's nice to meet you!\n\nBeing a software engineer is a fascinating field. What kind of software do you specialize in?  \n\nI'm always interested in learning more about different areas of software development.\n"

In [41]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config= config
)

AIMessage(content='Your name is Nageshwar.  \n\nI remember that you told me at the beginning of our conversation. 😊  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 89, 'total_tokens': 126, 'completion_time': 0.067272727, 'prompt_time': 0.00257538, 'queue_time': 0.256125969, 'total_time': 0.069848107}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4cc27f11-c4ea-4ef0-95b9-efede7213628-0', usage_metadata={'input_tokens': 89, 'output_tokens': 37, 'total_tokens': 126})

In [42]:
## Change the config ---> Changing the session_id
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config= config1
)
response.content

"As an AI, I don't have access to any personal information about you, including your name. If you'd like to tell me your name, I'd be happy to know!\n"

## Prompt Templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [43]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt= ChatPromptTemplate.from_messages(
    [
        ("system","You are a useful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="message")
    ]
)
chain= prompt | model

In [44]:
chain.invoke({"message":[HumanMessage(content="Hi my name is Nageshwar")]})

AIMessage(content="Hello Nageshwar, it's nice to meet you! \n\nI'm ready to assist you. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 33, 'total_tokens': 70, 'completion_time': 0.067272727, 'prompt_time': 0.00147694, 'queue_time': 0.25419742, 'total_time': 0.068749667}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--214a3a9b-330c-4c6d-aa92-15c3bec77686-0', usage_metadata={'input_tokens': 33, 'output_tokens': 37, 'total_tokens': 70})

In [45]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [46]:
config = {"configurable":{"session_id":"chat3"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Nageshwar")],
    config=config
)

response.content

"Hello Nageshwar, it's nice to meet you!\n\nHow can I be of service today?  I'm ready for your questions. 😊 \n"

In [49]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer all the questions to the best of your ability in {language}."
         ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt |model

In [55]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Nageshwar")],"language":"Hindi"})
response.content

'नमस्ते नागेश्वर! 👋 \n\nमुझे आपकी मदद करने में खुशी हो रही है।  आप क्या जानना चाहते हैं? 😄 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [56]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [74]:
config = {"configurable": {"session_id" :"chat4"}}
response =with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Nageshwar")],"language":"Hindi"},
    config = config
)
response.content

'प्रिय नागेश्वर,  \n\nआप कैसे हो? क्या मैं आपकी कोई मदद कर सकता हूँ? 😊  \n'

In [75]:
response =with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")],"language":"Hindi"},
    config = config
)
response.content

'आपका नाम नागेश्वर है। 😊 \n\n\n'

### Manage the conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

"trim_messages" helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [78]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45, 
    strategy="last",
    token_counter =model,
    include_system =True,
    allow_partial = False,
    start_on= "human"
)

messages = [
    SystemMessage(content="You're a good assistant."),
    HumanMessage(content="Hi, I am bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\AgenticAI\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PCC\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[SystemMessage(content="You're a good assistant.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [79]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages= itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What ice cream do i like?")],
    "language": "English"
    }
)

response.content

"As a large language model, I have no memory of past conversations and don't know your personal preferences. \n\nWhat's your favorite flavor of ice cream?  🍦\n"

In [82]:
response = chain.invoke(
    {
        "messages": messages +[HumanMessage(content="What math problem did i ask")],
        "language" : "English",
    }
)
response.content

'You asked "What\'s 2+2?"  😄  \n\n\n\nLet me know if you\'d like to try another one!\n'

In [83]:
## Lets wrap this in the message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable": {"session_id" :"chat5"}}

In [85]:
response = with_message_history.invoke(
    {
    "messages": messages + [HumanMessage(content="What is my name?")],
    "language": "English",
    },
    config = config,
)
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIs there a name you'd like to go by? 😊\n"